# Create a spreadsheet with the results

En este notebook se genera una hoja de calculo con los datos generados por los notebooks 
* 
* 
* 

In [43]:
from os.path import isfile

import numpy as np
import pandas as pd
import h5py

from spikelib.utils import datasets_to_array

In [44]:
nd_filter = 'nd2'
intensity_sta = nd_filter+'-255'
intensities_flash = ['{}_{:03d}'.format(nd_filter, intensity) for intensity in [50, 100, 150, 200, 255]]
experiment = 'MR-0092t2'
fanalysis = '../../data/processed_protocols/MR-0092t2_modified_analysis_of_protocols_150um_merge.hdf5'
fspikes = '../../data/sorting/MR-0092t2.result-1.hdf5'
outút_folder = '../../reports/spreadsheets/'

print('To export {} experiment to csv file'.format(experiment))

results = pd.DataFrame({})
    

To export MR-0092t2 experiment to csv file


## Scotopic

In [45]:
df = pd.read_csv('../../data/sync/MR-0092t2/event_list_MR-0092t2.csv')

protocol_names = ['scotopic_initial', 'scotopic_final']

for protocol_name in protocol_names:
    start_scotopic = df[df['protocol_name'].isin([protocol_name])]['start_event'].values[0]
    end_scotopic = (df[df['protocol_name'].isin([protocol_name])]['end_event']).values[0]
    scotopic_rate = {}
    with h5py.File(fspikes) as fscotopic: 
        for key, spikes in fscotopic['spiketimes'].items():
            spk = spikes[...]
            filter_spikes = spk < end_scotopic
            scotopic_rate[key] = spk[filter_spikes].size/(end_scotopic/20000.)

    scotopic_char = pd.Series(scotopic_rate, name='fr_'+protocol_name)
    results = pd.concat([results, scotopic_char],axis=1)

/home/cesar/Software/anaconda3/envs/spklib3/lib/python3.6/site-packages/ipykernel/__main__.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



## Photopic

## Temporal properties from the White Noise stimulus

In [46]:
# Source file
temp_char_group = '/sta/{}/temporal/char/'.format(intensity_sta)
valid_group = '/sta/{}/valid/'.format(intensity_sta)
with h5py.File(fanalysis, 'r') as panalysis:
    col_name = panalysis[temp_char_group].attrs['col_name'].split(',')
    
    data_char = {}
    valid = {}
    for key in panalysis[temp_char_group]:
        data_char[key] = panalysis[temp_char_group+key][...]
        valid[key] = panalysis[valid_group+key][...]
    
    wn_temp_char = pd.DataFrame(data_char, index=col_name).transpose()
    wn_temp_char['validsta'] = pd.Series(valid)


wn_temp_char['statype'].replace({1:'ON',0:'OFF'},inplace=True)

# Partial result
wn_col_name = {'peakamplitud': 'RF_PeakFrame_Amp',
               'peaktime': 'RF_PeakFrame',
               'peaktime2': 'RF_PeakFrame2',
               'biphasicindex': 'RF_BiphasicIndex',
               'zerocrossing': 'RF_ZeroCrossing',
               'statype': 'RF_STAtype',
               'validsta': 'RF_valid',
              }

wn_temp_char.rename(columns=wn_col_name,inplace=True)
results = pd.concat([results, wn_temp_char],axis=1)

## Temporal fit for WN

In [47]:
# Source file
temp_fit_pars_group = '/sta/{}/temporal/fit_params/'.format(intensity_sta)

with h5py.File(fanalysis, 'r') as panalysis:
    col_name = panalysis[temp_fit_pars_group].attrs['col_name'].split(',')
    
    fit_pars = {}
    for key in panalysis[temp_fit_pars_group]:
        fit_pars[key] = panalysis[temp_fit_pars_group+key][...]
    
    wn_temp_fit = pd.DataFrame(fit_pars, index=col_name).transpose()
    rename = ['RF_Fit_amp1', 'RF_Fit_amp2', 'RF_Fit_tau1', 'RF_Fit_tau2', 'RF_Fit_n']
    rename = {col_name[k]: rename[k] for k in range(len(col_name))}
    wn_temp_fit.rename(columns=rename,inplace=True)

print(wn_temp_fit.head())
results = pd.concat([results, wn_temp_fit],axis=1)

          RF_Fit_amp1  RF_Fit_amp2  RF_Fit_tau1  RF_Fit_tau2   RF_Fit_n
temp_0       0.000000     0.000000    -0.000000    -0.000000   0.000000
temp_1       0.345390     0.154075    -0.089396    -0.204000  10.238035
temp_10      0.258726     0.147207    -0.106037    -0.163156  13.030626
temp_100     0.024028     0.153281    -0.270798    -0.059177   9.486058
temp_101     0.000000     0.000000    -0.000000    -0.000000   0.000000


## Spatial properties from the White Noise stimulus

In [48]:
block_to_um = 50  # 12 pixel x 1 block x  4.1666 micrometers
spatial_char_group = '/sta/{}/spatial/char/'.format(intensity_sta)

with h5py.File(fanalysis, 'r') as panalysis:
    col_name = panalysis[spatial_char_group].attrs['col_name'].split(',')
    array, key = datasets_to_array(panalysis[spatial_char_group])
#     spatial_char = {key: panalysis[spatial_char_group+key][...] for key in panalysis[spatial_char_group]}

wn_spatial = pd.DataFrame(data=array, index=key, columns=col_name)
wn_spatial.columns

rename_columns = {
    'angle': 'RF_rotation',
    'semia': 'RF_semia',
    'semib': 'RF_semib',
    'x0': 'RF_x',
    'y0': 'RF_y',
    'snr': 'RF_snr',
    'frame_max': 'RF_frame_maxr'
}
wn_spatial.rename(columns=rename_columns, inplace=True)
wn_spatial['RF_semia'].iloc[:] = wn_spatial['RF_semia']*block_to_um
wn_spatial['RF_semib'].iloc[:] = wn_spatial['RF_semib']*block_to_um
wn_spatial['RF_x'].iloc[:] = wn_spatial['RF_x']*block_to_um
wn_spatial['RF_y'].iloc[:] = wn_spatial['RF_y']*block_to_um
wn_spatial['RF_area'] = wn_spatial['RF_semia'] * wn_spatial['RF_semib'] * np.pi
wn_spatial['RF_radmax'] = wn_spatial[['RF_semia','RF_semib']].max(axis=1)

results = pd.concat([results, wn_spatial],axis=1)   

# {'snr': 'RF_snr',
# 'exp': 'RF_exp',
# 'var_signal': 'RF_var_signal',
# 'var_noise': 'RF_var_noise',
# 'max_amp_raw': 'RF_max_amp_raw',
# 'min_amp_raw': 'RF_min_amp_raw',
# 'peak_amp1_norm': 'RF_PeakFrame_Amp_norm'}

## Linearity for the white noise stimulus

## Temporal propierties of the Flash stimulus

In [49]:
with h5py.File(fanalysis, 'r') as panalysis:
    for intensity in intensities_flash:
        nd = intensity.split('_')[1]
        flash_char_group = '/flash/{}/char/'.format(intensity)
        flash_type_group = '/flash/{}/type/'.format(intensity)
        col_name = panalysis[flash_char_group].attrs['col_name'].split(',')
        col_name = ['FL_{}_{}'.format(nd, k.strip()) for k in col_name]
        array, key = datasets_to_array(panalysis[flash_char_group])
        flash_char = pd.DataFrame(data=array, index=key, columns=col_name, )
        
        type_, key_type = datasets_to_array(panalysis[flash_type_group])
        flash_char['FL_{}_type'.format(nd)] = pd.Series(type_, index=key_type)
        flash_char['FL_{}_type'.format(nd)] = \
            flash_char['FL_{}_type'.format(nd)].map(
            {np.nan: 'Null', 0: 'Null', 1: 'On', 2: 'Off', 3: 'OnOff'},
        )
        results = pd.concat([results, flash_char],axis=1)   

## Sorting class

In [50]:
target_path = '../../data/sorting/target_units_MR-0092t2.result-1.json'
with open(target_path, 'r') as f:
    targets = json.loads(f.read())

target_units = []
for key in targets:
    target_units.extend(
        [*map(lambda k: ('temp_{}'.format(k), key), targets[key])]
    )
class_type = pd.DataFrame(target_units, columns=['name', 'class_type'])
class_type.set_index('name', inplace=True)
results = pd.concat([results, class_type],axis=1)   

/home/cesar/Software/anaconda3/envs/spklib3/lib/python3.6/site-packages/ipykernel/__main__.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



## Save results to CSV file

In [51]:
# resultFolder = '/home/cesar/exp/Chirp_data/results/'+experiment+'/'
file_name = outút_folder+experiment+'_'+nd_filter+'.csv'

results.to_csv(file_name, float_format='%.10f')
print('Saved in {}'.format(file_name))

results.head()

Saved in ../../reports/spreadsheets/MR-0092t2_nd2.csv


,fr_scotopic_initial,fr_scotopic_final,RF_PeakFrame,RF_PeakFrame2,RF_PeakFrame_Amp,RF_BiphasicIndex,RF_ZeroCrossing,RF_STAtype,RF_valid,RF_Fit_amp1,...,FL_255_decay_on,FL_255_decay_off,FL_255_resp_index_on,FL_255_resp_index_off,FL_255_sust_index_on,FL_255_sust_index_off,FL_255_peakresp_on,FL_255_peakresp_off,FL_255_type,class_type
temp_0,0.745888,2.897858,NaN,NaN,NaN,NaN,NaN,NaN,False,0.000000,...,0.000000,0.153084,0.000000,0.832359,0.000000,0.492399,0.000000,0.540638,Off,valid
temp_1,0.004929,2.445631,-0.087989,-0.208367,0.335456,0.451029,-0.143050,ON,True,0.345390,...,0.010006,0.000000,0.751642,0.000000,0.735465,0.000000,0.988772,0.000000,On,valid
temp_10,0.029573,0.859552,-0.099865,-0.184075,0.214373,0.512931,-0.143050,ON,True,0.258726,...,0.010006,0.000000,0.809126,0.000000,0.826974,0.000000,0.864268,0.000000,On,valid
temp_100,1.258480,3.456995,-0.059379,-0.270985,-0.153251,0.156789,-0.126316,OFF,True,0.024028,...,0.011006,0.069038,0.812106,0.760366,0.788744,0.666615,1.633333,1.062532,OnOff,valid
temp_101,0.023001,1.140419,-0.485289,-0.485289,NaN,NaN,-0.485289,OFF,True,0.000000,...,0.013007,0.000000,0.714843,0.000000,0.693601,0.000000,0.570679,0.000000,On,valid


In [52]:
import os

path_file = '../../reports/spreadsheets/'
files = [
    x for x in os.listdir(path_file) if x.startswith('MR-0092t2_nd')
]

df_list = []
for k in sorted(files):
    print(k)
    data = pd.read_csv(path_file+k)
    data['nd_filter'] = k.split('_')[1][:3]
    df_list.append(data)
df = pd.concat(df_list)

df_valid = df[df['class_type']=='valid']
df.to_csv(path_file+'MR-0092t2_allnd.csv')
df[df['class_type']=='valid'].to_csv(path_file+'MR-0092t2_allnd_valid.csv')

MR-0092t2_nd2.csv
MR-0092t2_nd3.csv
MR-0092t2_nd4.csv
MR-0092t2_nd5.csv
